In [6]:
import sqlite3
import pandas as pd
import os
import matplotlib
import numpy as np

In [3]:
# Connect to the database
db_connection = sqlite3.connect('mydatabase.db')
query = "SELECT * FROM box_scores"
query2 = "SELECT * FROM fixture_information"
query3 = "SELECT * FROM test_fixtures"
query4 = "SELECT * FROM test_fixtures_actuals"
df_box_scores = pd.read_sql_query(query, db_connection)
df_fixture_info = pd.read_sql_query(query2, db_connection)
df_test_fixtures = pd.read_sql_query(query3, db_connection)
df_test_fixtures_actuals = pd.read_sql_query(query4, db_connection)
db_connection.close()


In [10]:
#only missing of NA values in the dataframes is the handicap column in the df_test_fixtures_actuals
df_box_scores
df_fixture_info
df_test_fixtures
df_test_fixtures_actuals

,FixtureKey,Team,X2PM,X2PA,X3PM,X3PA,FTM,FTA,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,IsWinner,TeamHandicap
0,ID ST v E WASH 25-Feb-2023,2,9,19,11,31,12,15,8,25,15,3,4,19,23,63,0,-6.0
1,ID ST v E WASH 25-Feb-2023,1,13,27,10,28,15,20,9,24,15,7,3,11,19,71,1,6.0
2,WEB ST v IDAHO 25-Feb-2023,2,7,24,9,23,12,19,4,28,10,4,2,13,17,53,0,8.0
3,WEB ST v IDAHO 25-Feb-2023,1,16,35,9,23,8,10,6,32,11,8,1,8,16,67,1,-8.0
4,MONT v SAC ST 25-Feb-2023,1,18,30,8,23,14,20,7,16,12,7,2,7,18,74,1,-7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,NC CEN v HOWARD 25-Feb-2023,1,21,41,4,17,14,17,14,26,11,5,8,15,14,68,1,NaN
256,JACKST v TX SOU 25-Feb-2023,1,22,41,4,10,15,18,6,19,8,9,2,9,10,71,1,-1.5
257,JACKST v TX SOU 25-Feb-2023,2,24,46,5,14,6,6,9,20,8,6,2,12,16,69,0,1.5
258,BUCKNL v LAFAYE 25-Feb-2023,1,17,26,9,27,14,19,6,27,20,7,4,9,13,75,1,NaN


In [12]:
df_test_fixtures_actuals.isna()

,FixtureKey,Team,X2PM,X2PA,X3PM,X3PA,FTM,FTA,ORB,DRB,AST,STL,BLK,TOV,PF,PTS,IsWinner,TeamHandicap
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
256,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
257,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
258,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [4]:
df_box_scores.describe()

,Team,X2PM,X2PA,X3PM,X3PA,FTM,FTA,ORB,DRB,AST,STL,BLK,TOV,PF
count,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000
mean,1.500000,17.894385,35.849797,7.275409,21.566507,12.707551,17.729320,9.730446,24.799354,12.807987,6.402417,3.200570,12.598259,16.938147
std,0.500019,4.561192,7.168725,3.057644,5.980259,5.730298,7.285578,3.896482,4.959733,4.347827,2.881146,2.156794,4.004182,4.274538
min,1.000000,3.000000,11.000000,0.000000,2.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,0.000000,1.000000,4.000000
25%,1.000000,15.000000,31.000000,5.000000,17.000000,9.000000,12.000000,7.000000,21.000000,10.000000,4.000000,2.000000,10.000000,14.000000
50%,1.500000,18.000000,36.000000,7.000000,21.000000,12.000000,17.000000,9.000000,25.000000,12.000000,6.000000,3.000000,12.000000,17.000000
75%,2.000000,21.000000,40.000000,9.000000,25.000000,16.000000,22.000000,12.000000,28.000000,15.000000,8.000000,4.000000,15.000000,20.000000
max,2.000000,46.000000,72.000000,21.000000,54.000000,50.000000,65.000000,51.000000,48.000000,33.000000,22.000000,16.000000,37.000000,40.000000


In [8]:
df_box_scores_2 = pd.DataFrame()
df_box_scores_2['HomeTeam'] = df_box_scores['FixtureKey'].apply(lambda x:x.split(" v ")[0])
df_box_scores_2['AwayTeam'] = df_box_scores['FixtureKey'].apply(lambda x:x.split(" v ")[1])
df_box_scores_2['AwayTeam'] = df_box_scores_2['AwayTeam'].apply(lambda x:x.replace(x[-11:],""))
df_box_scores_2['Dates'] = df_box_scores['FixtureKey'].apply(lambda x: x.split(x[:-11])[1])
df_box_scores_2 = df_box_scores_2.set_index('Dates', drop=True)
for i in np.arange(2,len(df_box_scores.columns),1):
    df_box_scores_2[df_box_scores.columns[i]] = df_box_scores[df_box_scores.columns[i]].tolist()
df_box_scores_2['X2PM'].groupby(df_box_scores_2['HomeTeam']).mean()
df_box_scores_2['X2PM'].groupby(df_box_scores_2['AwayTeam']).mean()

In [10]:
df_box_scores_2.describe()

,X2PM,X2PA,X3PM,X3PA,FTM,FTA,ORB,DRB,AST,STL,BLK,TOV,PF
count,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000,13322.000000
mean,17.894385,35.849797,7.275409,21.566507,12.707551,17.729320,9.730446,24.799354,12.807987,6.402417,3.200570,12.598259,16.938147
std,4.561192,7.168725,3.057644,5.980259,5.730298,7.285578,3.896482,4.959733,4.347827,2.881146,2.156794,4.004182,4.274538
min,3.000000,11.000000,0.000000,2.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,0.000000,1.000000,4.000000
25%,15.000000,31.000000,5.000000,17.000000,9.000000,12.000000,7.000000,21.000000,10.000000,4.000000,2.000000,10.000000,14.000000
50%,18.000000,36.000000,7.000000,21.000000,12.000000,17.000000,9.000000,25.000000,12.000000,6.000000,3.000000,12.000000,17.000000
75%,21.000000,40.000000,9.000000,25.000000,16.000000,22.000000,12.000000,28.000000,15.000000,8.000000,4.000000,15.000000,20.000000
max,46.000000,72.000000,21.000000,54.000000,50.000000,65.000000,51.000000,48.000000,33.000000,22.000000,16.000000,37.000000,40.000000
